In [1]:
import requests
import random
from bs4 import BeautifulSoup as soup
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import webbrowser
from datetime import date

In [2]:
ratings = ['safe', 'questionable', 'explicit', 'any']
status_codes = {200: 'OK', 204: 'No Content', 400: 'Bad Request', 401: 'Unauthorized',
               403: 'Forbidden', 404: 'Not Found', 410: 'Gone', 420: 'Invalid Record',
               422: 'Locked', 423: 'Already Exist', 424: 'Invalid Parameters', 429: 'User Throttled',
               500: 'Internal Server Error', 502: 'Bad Gateway', 503: 'Service Unavailable'}

def space_to_underscore(string):
    return string.replace(' ', '_')

In [3]:
client = False # true if running in jupyter notebook, false if running in virtual machine
name = ""
full_name = "Hu Tao (Genshin Impact)"
day = 1
start_page = 1
end_page = 5
rating = 'safe'
no_rating = ''
source = 'https://danbooru.donmai.us'
max_width = 720
max_height = 360
today_date = date.today().strftime('%m/%d/%Y')
post_url = ""
img_url = ""
source_url = ""

In [4]:
# build query
tags = space_to_underscore(full_name)
if len(rating) != 0:
    tags += ' rating:{}'.format(rating)
if len(no_rating) != 0:
    tags += ' -rating:{}'.format(no_rating)
query = {'tags': tags}

In [5]:
def get_num_pages_images(source, query):
    num_pages = 0
    num_images = 0
    while True:
        query['page'] = num_pages + 1
        response = requests.get(source, params=query)
        articles = soup(response.text, 'html.parser').find_all('article')
        if len(articles) == 0:
            break
        num_pages += 1
        num_images += len(articles)
    return num_pages, num_images

def get_num_pages(source, query):
    return get_num_pages_images(source, query)[0]

def get_num_images(source, query):
    return get_num_pages_images(source, query)[1]

In [6]:
def get_post_ids(source, query, start_page, end_page):
    post_ids = []
    for page in range(start_page, end_page + 1):
        query['page'] = page
        response = requests.get(source, params=query)
        articles = soup(response.text, 'html.parser').find_all('article')
        for article in articles:
            post_ids.append(article['data-id'])
    return post_ids

In [7]:
post_ids = get_post_ids(source, query, start_page, end_page)

In [8]:
def get_random_post_id(post_ids):
    return random.choice(post_ids)

def get_random_post_url(post_ids):
    post_id = get_random_post_id(post_ids)
    return source + '/posts/' + post_id

def get_post_img_info(post_url):
    return soup(requests.get(post_url).text, 'html.parser').find(class_='image-container note-container')

def adjust_image_size(post_img_info):
    ori_width = int(post_img_info['data-width'])
    ori_height = int(post_img_info['data-height'])
    resize_ratio = min(max_width/ori_width, max_height/ori_height)
    disp_width = int(resize_ratio * ori_width)
    disp_height = int(resize_ratio * ori_height)
    return disp_width, disp_height

In [9]:
def get_image_widget(post_ids):
    global post_url, img_url, source_url
    post_url = get_random_post_url(post_ids)
    post_img_info = get_post_img_info(post_url)
    source_url = post_img_info['data-normalized-source']
    disp_width, disp_height = adjust_image_size(post_img_info)
    img_url = post_img_info['data-file-url']
    img = widgets.HTML('<p style="text-align:center;"><img src="{}" width="{}" height="{}"></img></p>'
                   .format(img_url, disp_width, disp_height))
    return img

img = get_image_widget(post_ids)

In [10]:
def get_header_widget():
    header = widgets.HTML('<h1 align="center">Daily {} posting #{}</h1> <h3 align="center"> ({}) </h3>'
                      .format(name, day, today_date))
    return header

header = get_header_widget()

In [11]:
def open_url(url):
    if client:
        webbrowser.open(url, new=2)
    else:
        with out:
            display(Javascript('window.open("{}");'.format(url)))

In [12]:
def view_post(obj):
    open_url(post_url)
view_post_btn = widgets.Button(description = 'view post', layout=widgets.Layout(height='auto', width='auto'))
view_post_btn.style.button_color = 'lightblue'
view_post_btn.on_click(view_post)

def view_source(obj):
    open_url(source_url)
view_source_btn = widgets.Button(description = 'view source', layout=widgets.Layout(height='auto', width='auto'))
view_source_btn.style.button_color = 'lightgreen'
view_source_btn.on_click(view_source)

def download(obj):
    download_url = soup(requests.get(post_url).text, 'html.parser').find(id="post-option-download").find('a')['href']
    open_url(download_url)
download_btn = widgets.Button(description = 'download', layout=widgets.Layout(height='auto', width='auto'))
download_btn.style.button_color = 'orange'
download_btn.on_click(download)

def refresh(obj):
    out.clear_output(wait=True)
    global day
    day += 1
    header = get_header_widget()
    img = get_image_widget(post_ids)
    with out:
        display(search_panel, header, img, panel_box)
refresh_btn = widgets.Button(description = 'refresh', layout=widgets.Layout(height='auto', width='auto'))
refresh_btn.style.button_color = 'yellow'
refresh_btn.on_click(refresh)

In [13]:
panel = widgets.TwoByTwoLayout(top_left=view_post_btn,
               top_right=view_source_btn,
               bottom_left=download_btn,
               bottom_right=refresh_btn,
               layout=widgets.Layout(height='auto', width='500px'))
panel_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center')
panel_box = widgets.HBox(children=[panel], layout=panel_layout)
out = widgets.Output()

In [14]:
search_chara_btn = widgets.Text(
    value='Hu Tao (Genshin Impact)',
    placeholder='Search Character',
    description='Character:',
    disabled=False
)

def search_chara(obj):
    global full_name
    full_name = search_chara_btn.value

search_chara_btn.on_submit(search_chara)

In [15]:
def make_checkboxes(choices):
    return [widgets.Checkbox(value=False, description=choice, disabled=False, indent=False, 
                            layout=widgets.Layout(width='auto', margin='10px')) for choice in choices]

In [16]:
flex_box_row = widgets.Layout(display='flex', flex_flow='row', align_items='center')

In [17]:
rating_text = widgets.HTML('<p top=50%>Ratings:</p>', layout=widgets.Layout(margin='0px 0px 0px 25px'))
rating_checkboxes = make_checkboxes(ratings[:-1])
rating_checkboxes[0].value = True # default to safe
rating_panel = widgets.HBox(children=[rating_text, *rating_checkboxes], 
                            layout=flex_box_row)

In [18]:
start_page_btn = widgets.IntText(value=start_page, description='Page:', indent=False,
                    layout=widgets.Layout(width='140px', margin='0px 0px 0px -20px'))
end_page_btn = widgets.IntText(value=end_page, description='To:', indent=False,
                    layout=widgets.Layout(width='140px', margin='0px 20px 0px -50px'))
page_panel = widgets.HBox([start_page_btn, end_page_btn], layout=flex_box_row)

In [19]:
def build_query_ratings():
    query_ratings = ''
    checkbox_values = [checkbox.value for checkbox in rating_checkboxes]
    if sum(checkbox_values) == 1:
        pick = [rating for i, rating in enumerate(ratings[:-1]) if checkbox_values[i]][0]
        query_ratings = 'rating:{}'.format(pick)
    elif sum(checkbox_values) == 2:
        unpick = [rating for i, rating in enumerate(ratings[:-1]) if not checkbox_values[i]][0]
        query_ratings = '-rating:{}'.format(unpick)
    #else rating:any
    return query_ratings 

In [20]:
def search_handler(obj):
    global full_name, post_ids
    full_name = search_chara_btn.value
    query_ratings = build_query_ratings()
    tags = space_to_underscore(full_name.lower()) + ' ' + query_ratings
    query = {'tags': tags}
    start_page = start_page_btn.value
    end_page = end_page_btn.value
    new_post_ids = get_post_ids(source, query, start_page, end_page)
    if len(new_post_ids) == 0:
        print(tags)
        print('no search result found')
        return
    post_ids = new_post_ids
    header = get_header_widget()
    img = get_image_widget(post_ids)
    out.clear_output(wait=True)
    with out:
        display(search_panel, header, img, panel_box)
search_btn = widgets.Button(description = 'Search', button_style = 'info',
                            layout=widgets.Layout(height='auto', width='auto'))
search_btn.on_click(search_handler)

search_panel = widgets.HBox([search_chara_btn, rating_panel, page_panel, search_btn], layout=flex_box_row, margin='20px')

In [21]:
with out:
    display(search_panel, header, img, panel_box)

In [22]:
out

Output()